In [3]:
import os 

os.getcwd()
# go back to root folder
os.chdir('../')
os.getcwd()

'/home/gfspet/ml-projects/kidney-disease'

In [17]:
# %cd research/

Data Ingestion Configs

In [4]:
import sys
sys.path.append('/home/gfspet/ml-projects/kidney-disease/src')
from dataclasses import dataclass
from pathlib import Path 
from cnn_classifier.constants import * 
from cnn_classifier.utils.common import read_yaml, create_directories


@dataclass(frozen=True) 
class DataIngestionConfig: 
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig: 
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir, 
            source_URL=config.source_URL, 
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

Data Ingestion Component

In [ ]:
import os 
import zipfile
import gdown
from cnn_classifier import logger


class DataIngestion:
    def __init__(self, config=DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str: 
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True) 
            # os.makedirs(zip_download_dir, exist_ok=True)
            logger.info(f'Downloading data from ({dataset_url}) to ({zip_download_dir})')
            
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)
            logger.info(f'Download completed')
            
        except Exception as e: 
            raise e
        
    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: 
            zip_ref.extractall(unzip_path)

Data Ingestion Pipeline

In [10]:
try:
    config = ConfigurationManager() 
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config) 
    data_ingestion.download_file()
    data_ingestion.extract_zip()
except Exception as e: 
    raise e

[ 2024-11-23 22:02:18,532 ] 21 cnn_classifier_logger - INFO - yaml file: config/config.yaml loaded successfully
[ 2024-11-23 22:02:18,536 ] 21 cnn_classifier_logger - INFO - yaml file: params.yaml loaded successfully
[ 2024-11-23 22:02:18,539 ] 33 cnn_classifier_logger - INFO - Created directory at: artifacts
[ 2024-11-23 22:02:18,543 ] 33 cnn_classifier_logger - INFO - Created directory at: artifacts/data_ingestion
[ 2024-11-23 22:02:18,553 ] 18 cnn_classifier_logger - INFO - Downloading data from (https://drive.google.com/file/d/1RMUrozOYrdxD40pcXJMCAPRy6kIrBrg3/view?usp=sharing) to (artifacts/data_ingestion/data.zip)


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RMUrozOYrdxD40pcXJMCAPRy6kIrBrg3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RMUrozOYrdxD40pcXJMCAPRy6kIrBrg3&confirm=t&uuid=10094a5d-ffd0-40c7-9630-47818f159ca1
To: /home/gfspet/ml-projects/kidney-disease/artifacts/data_ingestion/data.zip
100%|██████████| 112M/112M [00:20<00:00, 5.36MB/s] 

[ 2024-11-23 22:02:43,963 ] 23 cnn_classifier_logger - INFO - Download completed
